# Emotion recognition

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical  # Import to_categorical directly
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
import cv2
import os
from PIL import Image
from sklearn.metrics import classification_report
import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K



# Preprocessing

In [20]:
'''
lets' see which is the dimesnions of the images we have as input. 
We will take as example just the filder with pictures about the angry class'''

DATASET_DIR='dataset/Angry/'
s_dim1=1000000
s_dim2=1000000
for i in os.listdir(DATASET_DIR):
    img = cv2.imread(DATASET_DIR+i, cv2.IMREAD_COLOR) # imread returns un array numpy
    print(img.shape)
    if img is not None:
        if img.shape[0]<s_dim1:
            s_dim1=img.shape[0]
        if img.shape[1]<s_dim2:
            s_dim2=img.shape[1]

print(f'The minimum number of pixels in the long side of the image is {s_dim1}')
print(f'The minimum number of pixels in the short side of the image is {s_dim2}')


we can see that the shape is not equal for all the images, this suggests us that we will need to reshape them before training the model. <br>
Since we will need to reshape the images, if there are some iages whose dimensions are smaller than the reshaoed one, we will need to discard them so we don't risk <br>
strange outputs

In [8]:
'Let\'s say we want the size of the images to be (125x125), we will discard alal teh images where one of the two dimensions is saller than 125'

count_pre_erase=0
for i in os.listdir(DATASET_DIR):
    count_pre_erase+=1


for filename in os.listdir(DATASET_DIR):
    img_path = os.path.join(DATASET_DIR, filename)
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    if img is not None:
        height, width = img.shape[:2]
        if height < 125 or width < 125:
            os.remove(img_path)  # Delete the image

count_post_erase=0
for i in os.listdir(DATASET_DIR):
    count_post_erase+=1


print(f'Number of images before erasing: {count_pre_erase}')
print(f'Number of images after erasing: {count_post_erase}')


Number of images before erasing: 1313
Number of images after erasing: 1289


the amount of images that we loose is not too high. <br>
So let's suppose that this rules is valid for all the folders containing images for different emotions

In [2]:
'''
since augmenting the pixels in an image could end up in an undeesired result, 
we will eraase those images that have at least one of the two dimensions smaller than 125'''

EMOTION_DIR = 'dataset/'

# Loop over each emotion directory in the dataset folder
for emotion in os.listdir(EMOTION_DIR):
    folder = os.path.join(EMOTION_DIR, emotion)
    
    # Check if the folder variable is indeed a directory
    if os.path.isdir(folder):
        # Loop over each image file in the emotion directory
        for filename in os.listdir(folder):
            img_path = os.path.join(folder, filename)
            img = cv2.imread(img_path, cv2.IMREAD_COLOR)
            
            if img is not None:
                height, width = img.shape[:2]
                if height < 125 or width < 125:
                    os.remove(img_path)
                    print(f"Deleted {img_path} because its size was smaller than 125.")
            else:
                print(f"Failed to read image at path: {img_path}")

In [5]:
from PIL import Image, UnidentifiedImageError
import os

'''
for sake of consistency, we want all the images having the same format, 
the code below will convert all the images in the dataset to jpg format'''


# Define the list of class directories
classes = ['Angry', 'Happy', 'Neutral', 'Sad', 'Surprise']
EMOTION_DIR = 'dataset'  # Ensure this is the correct path to your dataset

# Iterate through each class directory
for c in classes:
    current_dir = os.path.join(EMOTION_DIR, c)
    print(f"Processing directory: {current_dir}")  # Debugging output

    # Check if the directory exists
    if not os.path.exists(current_dir):
        print(f"Directory not found: {current_dir}")
        continue

    # Iterate through each file in the current directory
    for f in os.listdir(current_dir):
        if f.endswith('.png'):  # Process only PNG files
            img_path = os.path.join(current_dir, f)

            # Check if the file exists
            if not os.path.exists(img_path):
                print(f"File not found: {img_path}")
                continue

            try:
                # Open the PNG image
                with Image.open(img_path) as png_image:
                    # Convert image to 'RGB' if it's 'P' or 'RGBA' (JPEG does not support transparency)
                    if png_image.mode in ('RGBA', 'LA', 'P'):
                        png_image = png_image.convert('RGB')

                    # Define the new file name with the JPEG extension
                    jpg_file = os.path.splitext(f)[0] + '.jpg'
                    jpg_path = os.path.join(current_dir, jpg_file)

                    # Save the image in JPEG format
                    png_image.save(jpg_path, 'JPEG')

                # Remove the original PNG file
                os.remove(img_path)

            except UnidentifiedImageError as e:
                print(f"Error opening image {f} in {current_dir}: {e}")
            except Exception as e:
                print(f"Unexpected error processing image {f} in {current_dir}: {e}")


Processing directory: dataset/Angry
Processing directory: dataset/Happy
Processing directory: dataset/Neutral
Processing directory: dataset/Sad
Processing directory: dataset/Surprise


# 1 approach

we will now implement 2 diferent approaches to solve our problem in two different ways.<br> The first one will be taking the images, opening them with openCV, flatten the associated numpy array and succesively store the flattened vectors into a csv file that will be evaluated by a classical Neural Network.<br> The second apaproach will be characterised by the use of CNN with convolution and maxpooling layers; taking as input the images as such.

In [ ]:
'''
In this piece of code we will create a csv file conatining for each row the vector of pixels of the image and the label.
We will be doing it for each of the classes we have in the dataset and finally we will print the elements we have for each class.'''

EMOTION_DIR = 'dataset/'
# set the nmae of the csv file
OUTFILE_NAME = "EMOTIONS.csv"
# set the scale of the images
SCALE = (125, 125)

# opening the csv file in the wrtie mode so that we can append the vector of pixels and the label
out = open(EMOTION_DIR+OUTFILE_NAME, "w")
# assignign to each class a number
classes = {"Angry":"1", "Happy":"0", "Neutral":"2", "Sad":"3", "Surprise":"4"}
# setting a counter to check how many images we have for each class
counter = {"Angry":0, "Happy":0, "Neutral":0, "Sad":0, "Surprise":0}

print("Letture di tutte le immagini da %s" % EMOTION_DIR)
print("Scrittura in %s" % OUTFILE_NAME)

for c in classes:
    
    current_dir = EMOTION_DIR+c

    for f in os.listdir(current_dir): # iteerating over all the files that are in each of the emotion directory

        img = cv2.imread(current_dir+"/"+f, cv2.IMREAD_GRAYSCALE) # reading the image in grayscale
        img = cv2.resize(img, SCALE) # resize the image
        img = img.flatten().astype(str) # trasformiamo l'immagine in un array di stringhe
        data = ",".join(img)+","+classes[c] # scriviamo prima il valore dei vari pixel e poi il label
        out.write(data+"\n")
        
        counter[c]+=1
        print(f'{c}, {counter[c]}')

out.close()
print("Immagini scritte: %s" %counter)

In [ ]:
# once we have created the csv containing the values of the pixels and the label of each pictures we can import it in a dataframe
df = pd.read_csv(EMOTION_DIR+OUTFILE_NAME, header=None) # we don't have a header in the csv file
df.shape
'''
as we can see we have 14634 rows, that is the sum of each value of the counter we set in the chunk before,
 and 15626 columns that is equal to 125x125 the amount of pixels in the image, plus the column of the label'''

In [ ]:
'''
Preparing the data for the model.'''

# we split the dataframe in two parts, the first one containing all the columns except the last one, that is the label.
X = df.iloc[:,:-1]
y = df.iloc[:,-1:]
# we split the dataset in train and test set, the test set will be the 30% of the dataset, we set shuffle to True so that the data will be shuffled before splitting,
# and we set stratify to y so that the distribution of the labels in the train and test set will be the same.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, stratify=y, random_state=42)

encoder = LabelEncoder()
y_train_t = encoder.fit_transform(y_train)
y_test_t=encoder.transform(y_test)
y_train = to_categorical(y_train_t)
y_test = to_categorical(y_test_t)

# we finally divide the values by 255 so that we have all the values between 0 and 1, we normalize them.
X_train/=255
X_test/=255

In [ ]:
'''
now that the preprocessing part has been finished we can proceed with the traing of the model. We will set and train a classical neural netowrk.'''

# Creiamo la Rete Neurale

model1 = Sequential()
model1.add(Dense(5121, activation='relu', input_dim=X_train.shape[1]))
model1.add(Dropout(0.3)) # Dropout to avoid overfitting
model1.add(Dense(2570, activation='relu'))
model1.add(Dropout(0.3))
model1.add(Dense(2570, activation='relu'))
model1.add(Dropout(0.3))
model1.add(Dense(1285, activation='relu'))
model1.add(Dropout(0.3))
model1.add(Dense(1285, activation='relu'))
model1.add(Dropout(0.3))
model1.add(Dense(5, activation='softmax'))

model1.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

model1.fit(X_train, y_train, epochs=200, batch_size=100)

# Evaluating the model
metrics_train1 = model1.evaluate(X_train, y_train, verbose=0)
metrics_test1 = model1.evaluate(X_test, y_test, verbose=0)

print("Train Accuracy = %.4f - Train Loss = %.4f" % (metrics_train1[1], metrics_train1[0]))
print("Test Accuracy = %.4f - Test Loss = %.4f" % (metrics_test1[1], metrics_test1[0]))

# Saving the model
model1.save("emotions_ANN.h5")

# 2 approach

## Image augmentation

In [6]:
import shutil

'''
We have some classes that have a number of images that is much smaller than the others... too small to fit a good multiclass classification model.
So aim so is the one of augmenting the images of the less populated classes by rotating, flipping and zooming the images.
The very last step will be the one of merging the augmented images with the original ones and store them in a new folder.'''



def merge_directories(sources, destination):
    """
    Merge multiple source directories into a single destination directory.
    Files from source directories are copied into the destination directory.
    Note: This may overwrite files in the destination if filenames conflict.
    
    Parameters:
    - sources: List of paths to source directories.
    - destination: Path to the destination directory.
    """
    # Create the destination directory if it doesn't exist
    if not os.path.exists(destination):
        os.makedirs(destination)

    # Copy files from each source directory to the destination
    for source in sources:
        for filename in os.listdir(source):
            source_file = os.path.join(source, filename)
            if os.path.isfile(source_file):  # Ensure it's a file
                shutil.copy2(source_file, destination)  # This may overwrite files

In [9]:
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
import os

# set the categories we want to augment
categories = ['Angry', 'Surprise']
# set by how much we want to augment the images
categories_dict={'Angry':2, 'Surprise':3}


'''
we will now merge the two directories (for each of the augmented emotion) in a single one, 
after performing this we can copy the remaining three folder in the final_folders directory'''

for category in categories:
    # Example for a single category that needs augmentation
    category = category
    source_directory = os.path.join('dataset', category)
    augmented_directory = os.path.join('dataset/augmented_train', category)
    os.makedirs(augmented_directory, exist_ok=True)  # Ensure the target directory exists

    datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
    )

    # Assume you've determined each image needs to be augmented 5 times
    augment_factor = categories_dict[category]

    for filename in os.listdir(source_directory):
        if filename.endswith(".jpg"):  # Ensure to process only jpg images or adjust as needed
            img_path = os.path.join(source_directory, filename)
            img = load_img(img_path)
            x = img_to_array(img)
            x = x.reshape((1,) + x.shape)

            # Generate and save augmented images
            i = 0
            for batch in datagen.flow(x, batch_size=1, save_to_dir=augmented_directory, save_prefix='aug', save_format='jpg'):
                i += 1
                if i >= augment_factor:
                    break  # Stop after generating the desired number of augmented images
    
    # usage
    sources_1=os.path.join('dataset', category)
    sources_2=os.path.join('dataset/augmented_train', category)
    sources = [sources_1, sources_2]
    destination = os.path.join('dataset/final_folders', category)
    merge_directories(sources, destination)

In [10]:
''' now for seek of clarity let's count the elements of each folder'''
# counting elements in each folder containied in the final folders directory
counter = {"Angry":0, "Happy":0, "Neutral":0, "Sad":0, "Surprise":0}

base_dir = "dataset/final_folders"
for elem in os.listdir(base_dir):
    path = os.path.join(base_dir, elem)
    if os.path.isdir(path):  # Check if it's a directory
        for f in os.listdir(path):
            counter[elem] += 1

print(counter)

{'Angry': 3415, 'Happy': 3402, 'Neutral': 3941, 'Sad': 3858, 'Surprise': 3649}


## CNN

In [13]:
'''
Once everything has been set correctly we can proceed with the training of the model.
We will use a CNN model, ImageDataGenerator to load the images from the directory and the flow_from_directory method to load the images from the directory.'''

DATASET_DIR = "dataset/final_folders"
BATCH_SIZE = 200

# since we have a quite consistent number of images we cannot load them all in memory, 
# so we will use the flow_from_directory method to load the images from the directory.

datagen = ImageDataGenerator(
        validation_split = 0.3,
        rescale = 1./255,
        horizontal_flip = True, 
        zoom_range = 0.2,
        brightness_range = [1,2]
)

train_generator = datagen.flow_from_directory( # flow from directory reads the images from the directory and returns a batch of images
        DATASET_DIR,
        target_size = (125, 125), # resize the images to 125x125
        batch_size = BATCH_SIZE,
        class_mode = "categorical",
        subset = "training"
)


test_generator = datagen.flow_from_directory(
        DATASET_DIR,
        target_size = (125, 125),
        batch_size = BATCH_SIZE,
        class_mode = "categorical",
        subset = "validation",
        shuffle=False
)

print(train_generator.class_indices)

# since can be usefull to have paramters different from the accuracy to evaluate the model we will define them here

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


# let's implement the model

model = Sequential()
model.add(Conv2D(filters=64, kernel_size=4, padding="same", activation="relu", input_shape=(125, 125, 3)))
model.add(MaxPooling2D(pool_size=4, strides=4))
model.add(Dropout(0.3))
model.add(Conv2D(filters=32, kernel_size=4, padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=4, strides=4))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(5, activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy", f1_m, precision_m, recall_m])
model.fit(train_generator, epochs=100)

Found 12784 images belonging to 5 classes.
Found 5477 images belonging to 5 classes.
{'Angry': 0, 'Happy': 1, 'Neutral': 2, 'Sad': 3, 'Surprise': 4}
Epoch 1/100
64/64 [==============================] - 55s 847ms/step - loss: 1.6095 - accuracy: 0.2295 - f1_m: 4.4753e-04 - precision_m: 0.0179 - recall_m: 2.3437e-04
Epoch 2/100
64/64 [==============================] - 56s 861ms/step - loss: 1.4797 - accuracy: 0.3561 - f1_m: 0.0691 - precision_m: 0.5389 - recall_m: 0.0383
Epoch 3/100
64/64 [==============================] - 57s 886ms/step - loss: 1.3584 - accuracy: 0.4280 - f1_m: 0.2072 - precision_m: 0.6228 - recall_m: 0.1259
Epoch 4/100
64/64 [==============================] - 58s 896ms/step - loss: 1.2837 - accuracy: 0.4699 - f1_m: 0.2977 - precision_m: 0.6545 - recall_m: 0.1937
Epoch 5/100
64/64 [==============================] - 59s 907ms/step - loss: 1.2225 - accuracy: 0.4985 - f1_m: 0.3617 - precision_m: 0.6836 - recall_m: 0.2472
Epoch 6/100
64/64 [==============================] - 

In [14]:
# Evaluating the model
metrics_train = model.evaluate(train_generator)
metrics_test = model.evaluate(test_generator)

print("Train Accuracy = %.4f - Train Loss = %.4f - Train F1= %.4f - Train precision= %.4f - Train recall= %.4f" % (metrics_train[1], metrics_train[0], metrics_train[2], metrics_train[3], metrics_train[4]))
print("Test Accuracy = %.4f - Test Loss = %.4f- Test F1= %.4f - Test precision= %.4f - Test recall= %.4f" % (metrics_test[1], metrics_test[0], metrics_test[2], metrics_test[3], metrics_test[4]))

28/28 [==============================] - 23s 809ms/step - loss: 0.5090 - accuracy: 0.8001 - f1_m: 0.7903 - precision_m: 0.8416 - recall_m: 0.7458
Train Accuracy = 0.8816 - Train Loss = 0.3617 - Train F1= 0.8668 - Train precision= 0.9164 - Train recall= 0.8225
Test Accuracy = 0.8001 - Test Loss = 0.5090- Test F1= 0.7903 - Test precision= 0.8416 - Test recall= 0.7458


In [15]:
# # Ensure the test_generator is not shuffling data
# test_generator.shuffle = False

# Make predictions
y_pred = model.predict(test_generator, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

# Extract true labels from the generator
y_true = test_generator.classes

# Calculate accuracy (optional, for verification)
accuracy = np.mean(y_pred_bool == y_true)
print(f"Calculated Accuracy: {accuracy:.4f}")

# Generate a confusion matrix (optional, for further verification)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_true, y_pred_bool)
print(cm)

# Generate classification report
print(classification_report(y_true, y_pred_bool, target_names=test_generator.class_indices.keys()))


28/28 [==============================] - 22s 786ms/step
Calculated Accuracy: 0.8008
[[907  16  39  50  12]
 [ 20 899  48  45   8]
 [ 39  50 846 239   8]
 [ 76  66 268 740   7]
 [ 26  18  34  22 994]]
              precision    recall  f1-score   support

       Angry       0.85      0.89      0.87      1024
       Happy       0.86      0.88      0.87      1020
     Neutral       0.69      0.72      0.70      1182
         Sad       0.68      0.64      0.66      1157
    Surprise       0.97      0.91      0.94      1094

    accuracy                           0.80      5477
   macro avg       0.81      0.81      0.81      5477
weighted avg       0.80      0.80      0.80      5477



In [16]:
# let's save the model
model.save("model_CNN_final.h5")

/Users/leo/miniconda3/envs/tf_macos/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Application

In [17]:
import cv2
from keras import backend as K
from tensorflow.keras.models import load_model


SCALE = (125, 125)
## since our model uses some metrics that are not build by default in keras, we need to redefine them before loading the model

# Define custom metrics
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

# Load the model with custom metrics
custom_objects = {'f1_m': f1_m, 'precision_m': precision_m, 'recall_m': recall_m}
model = load_model('model_CNN_final.h5', custom_objects=custom_objects)


In [19]:
# Assuming 'model' is already updated for multiclass classification with 6 output units using softmax activation
# AOpen the webcam
cap = cv2.VideoCapture(1)

if(not cap.read()[0]):
    print("Webcam non è disponible")
    exit(0)

# Frame just the faces using the hardcascade calssifier
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

# Define class names based on your classes. Adjust these names as per your actual classes.
class_names = ["Angry", "Happy", "Neutral", "Sad", "Surprise"]

while(cap.isOpened()):
    _, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    rects = face_cascade.detectMultiScale(gray, 1.1, 15)

    for rect in rects:
        img = frame[rect[1]:rect[1]+rect[3], rect[0]:rect[0]+rect[2]]
        small_img = cv2.resize(img, SCALE)
        small_img = cv2.cvtColor(small_img, cv2.COLOR_BGR2RGB)

        x = small_img.astype(float) / 255.
        x = x.reshape(1, SCALE[0], SCALE[1], 3)  # Reshape for the model

        predictions = model.predict(x)
        predicted_class_index = np.argmax(predictions)  # Get the index of the highest probability
        predicted_class_name = class_names[predicted_class_index]  # Map index to class name
        probability = round(np.max(predictions) * 100, 1)  # Get the highest probability

        cv2.rectangle(frame, (rect[0], rect[1]), (rect[0]+rect[2], rect[1]+rect[3]), (0,255,0), 2)
        cv2.rectangle(frame, (rect[0], rect[1]-30), (rect[0]+190, rect[1]), (0,255,0), cv2.FILLED)
        cv2.putText(frame, predicted_class_name+" ("+str(probability)+"%)", (rect[0]+7, rect[1]-2), cv2.FONT_HERSHEY_PLAIN, 1.4, (255,255,255), 2)

    cv2.imshow("Classification", frame)

    # press q to exit and end the session
    if(cv2.waitKey(1) == ord("q")):
        break


1/1 [==============================] - 0s 9ms/step
